In [1]:
import cx_Oracle as oci
import pandas as pd
from datetime import datetime, timedelta
import requests

from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
#######
#0. 자료추출(온라인)
###

url = 'http://www.tmon.co.kr/deal/4237136106'
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')
driver = webdriver.Chrome('C:/Users/MAEIL/Downloads/chromedriver/chromedriver.exe', options=options)

# Web parsing
driver.get(url)
driver.implicitly_wait(5)

bsObj = BeautifulSoup(driver.page_source, "lxml")

saleCount = driver.find_elements_by_xpath('//*[@id="view-default-scene-default"]/section[1]/div[3]/article[1]/div[3]/p[1]/span/span')[0].text.replace('개 구매','')
saleCount = int(saleCount)

driver.close()

In [3]:
# 중간결과 확인 : 온라인 수집정보
saleCount

595

In [4]:
#######
#1. 자료추출(Oracel SQL)
###

# DB 접속
con_text = open("C:/conn_info.txt", 'r', encoding='utf8')
con_text = con_text.read()
conn = oci.connect(con_text)

df_use1 = pd.read_sql("select count(*) from TPR_PRMT_CRD_INFO where crd_kind = '432' and actv_dtm is not null", con = conn ).values
df_use2 = pd.read_sql("select count(*) from TPR_PRMT_CRD_INFO where crd_kind = '433' and actv_dtm is not null", con = conn ).values
df_use3 = pd.read_sql("select count(*) from TPR_PRMT_CRD_INFO where crd_kind = '434' and actv_dtm is not null", con = conn ).values

sale_cnt = df_use1 + df_use2 + df_use3
sale_amt = df_use1*3 + df_use2*5 + df_use3*10

# DB 접속종료
conn.close()

#발송내용 셋팅
s_dt = (datetime.today()- timedelta(0)).strftime('%m/%d, %H:%M')

text1 = "**CJ 기프트카드 판매현황**\n" 
text2 = "- 티몬  판매량 : {}개 (판매율 {}%)\n".format(saleCount,round(int(saleCount)/1000*100,1))
text3 = "- MaeilDo 등록 : {}개 (등록율 {}%)\n".format(int(sale_cnt),round(int(sale_cnt)/int(saleCount)*100,1))
text4 = "- 총 판매 금액 : {}만원 ({}기준)\n\n".format( int(sale_amt), str(s_dt) )
text5 = "**권종별 등록 내용**\n"
text6 = "-  3만원 권종 : {} 개\n".format( int(df_use1) )
text7 = "-  5만원 권종 : {} 개\n".format( int(df_use2) )
text8 = "- 10만원 권종 : {} 개".format( int(df_use3) )

text = text1 + text2 +text3 + text4 +text5 +text6 + text7 +text8 

In [5]:
int(sale_amt)

1958

In [10]:
#######
# 2. 텔레그램 셋팅/발송
###

#발송 대상자
chat_id = ("826340134",  # 박경하 
           "61792877" )  # 배유미  
#            "838970099")  # 김소영  

# 발송 내용
chat_text = text

# maeildo_bot 고유 ID
API_TOKEN = '**********************************'  

# 텔레그램 발송 API 실행
for chat_id in chat_id:
    url = "https://api.telegram.org/bot%s/sendMessage?chat_id=%s&text=%s" % (API_TOKEN, chat_id,  chat_text)
    requests.get(url)      #텔레그램 발송


In [8]:
exit(0)

In [2]:
# url

In [179]:
#결과물 예시
print(text)

**CJ 기프트카드 판매현황**
- 티몬  판매량 : 125개 (판매율 12.5%)
- MaeilDo 등록 : 64개 (등록율 51.2%)
- 총 판매 금액 : 361만원 (09/03, 13:33기준)

**권종별 등록 내용**
-  3만원 권종 : 22 개
-  5만원 권종 : 25 개
- 10만원 권종 : 17 개
